## heart예제 (딥러닝 분류)

딥러닝 분류 : f1 score, confusion matrix 필수

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import tensorflow as tf
import kerastuner as kt
from keras.models import Sequential
from keras.layers import Dense,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/var/folders/3j/vdn2jmw56jl86z3vlf17czcw0000gn/T/ipykernel_1849/1639837430.py:14: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### CSV 파일 로딩하기

In [7]:
file_path = './heart.csv'
data = pd.read_csv(file_path)

data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [8]:
# 데이터 확인
print("데이터 형태:", data.shape)
print("\n컬럼 정보:\n", data.columns)
print("\n클래스 분포:\n", data["target"].value_counts())

# 결측치 확인
print("=== 결측치 현황 ===")
print(data.isnull().sum())

데이터 형태: (303, 14)

컬럼 정보:
 Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

클래스 분포:
 target
1    165
0    138
Name: count, dtype: int64
=== 결측치 현황 ===
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64


In [9]:
# 레이블 분포 확인
print("\n=== 레이블 분포 ===")
print(data['target'].value_counts())


=== 레이블 분포 ===
target
1    165
0    138
Name: count, dtype: int64


### 데이터 전처리

In [11]:
# 특성(X)과 레이블(y) 분리
X = data.drop("target", axis=1)
y = data["target"]

# 훈련 및 테스트 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# 분류기 초기화
classifiers = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}

# 모델 학습 및 평가
results = []
for name, clf in classifiers.items():
    clf.fit(X_train, y_train) # 모델 학습
    y_pred = clf.predict(X_test) # 테스트 데이터 예측
    acc = accuracy_score(y_test, y_pred) # 정확도 계산
    cm = confusion_matrix(y_test, y_pred) # 혼동 행렬 계산 (TP|FN|FP|TN)
    cr = classification_report(y_test, y_pred) # 분류 보고서
    f1 = f1_score(y_test, y_pred) # f1값 계산
    results.append((name, acc, cm, cr, f1)) # 결과를 리스트에 저장

# 결과 출력
print("\n=== 모델 성능 비교 ===")
for name, acc, cm, cr, f1 in results:
    print(f"\n======={name}=======")
    print(f"Accuracy: {acc:.4f}")
    print(f"f1_score: {f1}")
    print(cr)
    print("Confusion Matrix:")
    print(cm)


=== 모델 성능 비교 ===

=======Random Forest=======
Accuracy: 0.8525
f1_score: 0.8615384615384616
              precision    recall  f1-score   support

           0       0.86      0.83      0.84        29
           1       0.85      0.88      0.86        32

    accuracy                           0.85        61
   macro avg       0.85      0.85      0.85        61
weighted avg       0.85      0.85      0.85        61

Confusion Matrix:
[[24  5]
 [ 4 28]]

=======Decision Tree=======
Accuracy: 0.8361
f1_score: 0.8333333333333334
              precision    recall  f1-score   support

           0       0.79      0.90      0.84        29
           1       0.89      0.78      0.83        32

    accuracy                           0.84        61
   macro avg       0.84      0.84      0.84        61
weighted avg       0.84      0.84      0.84        61

Confusion Matrix:
[[26  3]
 [ 7 25]]

=======Logistic Regression=======
Accuracy: 0.8852
f1_score: 0.8923076923076924
              precision

### 딥러닝 작업 : MLP

In [14]:
#원-핫 인코딩
y_train_onehot = to_categorical(y_train - 1)  # 훈련 레이블만 인코딩
y_test_onehot = to_categorical(y_test - 1)    # 테스트 레이블 변환

In [15]:
# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 훈련 데이터로 학습
X_test_scaled = scaler.transform(X_test)        # 테스트 데이터 변환

In [16]:
# 데이터셋 Shape 확인
print("\n=== 데이터셋 Shape ===")
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_onehot shape:", y_train_onehot.shape)
print("y_test_onehot shape:", y_test_onehot.shape)


=== 데이터셋 Shape ===
X_train_scaled shape: (242, 13)
X_test_scaled shape: (61, 13)
y_train_onehot shape: (242, 1)
y_test_onehot shape: (61, 1)


### 딥러닝 모델

In [18]:
# MLP 신경망 정의
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # 입력층 → 은닉층1 (64개 뉴런)
    Dense(32, activation='relu'),  # 은닉층1 → 은닉층2 (32개 뉴런)
    Dense(1, activation='sigmoid')  # 출력층 (시그모이드 활성화)
])

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam 옵티마이저 (학습률 0.001)
    loss='binary_crossentropy',  # 이진 교차 엔트로피 손실
    metrics=['accuracy']  # 정확도 추적
)

# 검증 세트 분리 (훈련 데이터의 20%)
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, 
    test_size=0.2, 
    random_state=42
)

# 학습 (배치 크기 32, 50 에포크)
history = model.fit(
    X_train_sub, y_train_sub,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    shuffle=True,
    verbose=1  # 학습 과정 출력
)

/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-15 00:12:18.669479: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-04-15 00:12:18.669497: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-15 00:12:18.669503: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-15 00:12:18.669514: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-15 00:12:18.669522: I tensorflow/core/common_runtime/pluggable_device/plugga

Epoch 1/50


2025-04-15 00:12:18.878626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.3656 - loss: 32.2525 - val_accuracy: 0.3878 - val_loss: 10.6002
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4736 - loss: 9.6953 - val_accuracy: 0.4082 - val_loss: 11.4276
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5059 - loss: 6.8505 - val_accuracy: 0.3878 - val_loss: 7.1398
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3592 - loss: 7.2538 - val_accuracy: 0.4694 - val_loss: 7.7424
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3110 - loss: 7.5830 - val_accuracy: 0.3673 - val_loss: 5.8897
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4164 - loss: 5.3245 - val_accuracy: 0.4082 - val_loss: 7.4254
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4638 - loss: 4.8694 - val_accuracy: 0.3878 - val_loss: 5.0950
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3672 - loss: 5.7692 - val_accuracy: 0.4286 - val_loss: 5.0384
Epoch 9/50


In [19]:
# 테스트 평가
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
y_pred_proba = model.predict(X_test)  # 확률 예측
y_pred_dl = (y_pred_proba > 0.5).astype(int).flatten()  # 0.5 기준 이진 분류
f1_dl = f1_score(y_test, y_pred_dl)
print("\n=== 딥러닝 모델 (TensorFlow) ===")
print(f"F1 Score: {f1_dl:.4f}")
print(f"정확도: {test_acc:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dl))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

=== 딥러닝 모델 (TensorFlow) ===
F1 Score: 0.8136
정확도: 0.8197
Confusion Matrix:
[[26  3]
 [ 8 24]]


In [20]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# # 데이터 텐서 변환
# X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
# X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# # 검증 세트 분리 (20% of training data)
# X_train_sub, X_val, y_train_sub, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=42
# )

# # DataLoader 생성
# train_dataset = TensorDataset(
#     torch.tensor(X_train_sub, dtype=torch.float32),
#     torch.tensor(y_train_sub.values, dtype=torch.float32).view(-1, 1)
# )
# val_dataset = TensorDataset(
#     torch.tensor(X_val, dtype=torch.float32),
#     torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)
# )

# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32)

# # 신경망 정의
# class HeartDiseaseClassifier(nn.Module):
#     def __init__(self, input_dim):
#         super(HeartDiseaseClassifier, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 64)  # 입력 차원 동적 설정
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 1)         # 출력 뉴런 1개
        
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))         # ReLU 활성화
#         x = torch.relu(self.fc2(x))
#         x = torch.sigmoid(self.fc3(x))      # 시그모이드 추가
#         return x
# # 모델 설정
# model = HeartDiseaseClassifier(input_dim=X_train.shape[1])
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # 학습 루프
# num_epochs = 50
# for epoch in range(num_epochs):
#     # Training
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
    
#     # Validation
#     model.eval()
#     val_loss = 0.0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             outputs = model(inputs)
#             val_loss += criterion(outputs, labels).item()
#             predicted = (outputs > 0.5).float()
#             correct += (predicted == labels).sum().item()
#             total += labels.size(0)
    
#     # 에포크당 결과 출력 (선택 사항)
#     print(f"Epoch {epoch+1}/{num_epochs} | "
#           f"Train Loss: {train_loss/len(train_loader):.4f} | "
#           f"Val Loss: {val_loss/len(val_loader):.4f} | "
#           f"Val Acc: {correct/total:.4f}")

# # 테스트 평가
# model.eval()
# with torch.no_grad():
#     y_pred_proba = model(X_test_tensor)
#     y_pred_dl = (y_pred_proba > 0.5).float().numpy().flatten()

# accuracy = accuracy_score(y_test, y_pred_dl)
# cm_dl = confusion_matrix(y_test, y_pred_dl)

# print("\n=== 딥러닝 모델 (PyTorch) ===")
# print(f"정확도: {accuracy:.4f}")
# print("Confusion Matrix:")
# print(cm_dl)

# # 머신러닝 vs 딥러닝 결과 비교
# print("\n=== 최종 성능 비교 ===")
# print("SVM:", accuracy_score(y_test, models["SVM"].predict(X_test)))
# print("PyTorch DL:", accuracy)
